In [1]:
import json

In [3]:
data = json.load(open('./keys.json', 'r'))

In [5]:
data['key']

'c7da74716ccd472198daa6c6ddb84ca4'

In [51]:
import json

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

loca_image_path = './img/001_見積書_タテ型_通常版.png'

keys = json.load(open('./keys.json', 'r'))
subscription_key = keys["key"]
endpoint = keys["endpoint"]

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [52]:
local_image_path = loca_image_path
local_image = open(local_image_path, "rb")

print("===== Read File - local =====")
# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(local_image,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()
'''
END - Read File - local
'''

print("End of Computer Vision quickstart.")

===== Read File - local =====
見 積 書
[254.0, 64.0, 342.0, 64.0, 342.0, 84.0, 254.0, 84.0]
サンプル株式会社 御中
[39.0, 101.0, 219.0, 101.0, 219.0, 120.0, 39.0, 120.0]
No
[456.0, 101.0, 472.0, 101.0, 472.0, 112.0, 456.0, 111.0]
1001
[527.0, 100.0, 554.0, 100.0, 554.0, 112.0, 527.0, 112.0]
見積日
[437.0, 118.0, 472.0, 118.0, 471.0, 131.0, 437.0, 131.0]
2022/4/30
[497.0, 118.0, 555.0, 119.0, 554.0, 132.0, 497.0, 131.0]
下記のとおり、御見積もり申し上げます。
[40.0, 156.0, 255.0, 156.0, 255.0, 169.0, 40.0, 169.0]
サンブル株式会社
[398.0, 156.0, 490.0, 156.0, 490.0, 170.0, 398.0, 170.0]
件名
[68.0, 176.0, 91.0, 176.0, 91.0, 189.0, 68.0, 188.0]
サンブルブロジェクト
[125.0, 175.0, 240.0, 175.0, 240.0, 188.0, 125.0, 188.0]
₸100-0001
[397.0, 175.0, 459.0, 176.0, 459.0, 188.0, 397.0, 187.0]
2022/4/30
[123.0, 195.0, 182.0, 195.0, 182.0, 207.0, 123.0, 207.0]
東京都千代田区千代田1-1-1
[397.0, 194.0, 543.0, 194.0, 543.0, 208.0, 397.0, 208.0]
支払条件
[56.0, 213.0, 104.0, 214.0, 104.0, 227.0, 56.0, 226.0]
月末締翌月末払
[123.0, 214.0, 205.0, 213.0, 205.0, 226.0, 123.0, 226.

In [12]:
read_response

In [13]:
read_response.headers["Operation-Location"]

'https://20231015myvision.cognitiveservices.azure.com/vision/v3.2/read/analyzeResults/87c02734-c628-4d3a-9edc-f058458402a0'

In [14]:
read_result

In [18]:
text_result.lines

In [53]:
result_list = []

if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            result_list.append([line.text, line.bounding_box])

In [54]:
result_list

[['見 積 書', [254.0, 64.0, 342.0, 64.0, 342.0, 84.0, 254.0, 84.0]],
 ['サンプル株式会社 御中', [39.0, 101.0, 219.0, 101.0, 219.0, 120.0, 39.0, 120.0]],
 ['No', [456.0, 101.0, 472.0, 101.0, 472.0, 112.0, 456.0, 111.0]],
 ['1001', [527.0, 100.0, 554.0, 100.0, 554.0, 112.0, 527.0, 112.0]],
 ['見積日', [437.0, 118.0, 472.0, 118.0, 471.0, 131.0, 437.0, 131.0]],
 ['2022/4/30', [497.0, 118.0, 555.0, 119.0, 554.0, 132.0, 497.0, 131.0]],
 ['下記のとおり、御見積もり申し上げます。',
  [40.0, 156.0, 255.0, 156.0, 255.0, 169.0, 40.0, 169.0]],
 ['サンブル株式会社', [398.0, 156.0, 490.0, 156.0, 490.0, 170.0, 398.0, 170.0]],
 ['件名', [68.0, 176.0, 91.0, 176.0, 91.0, 189.0, 68.0, 188.0]],
 ['サンブルブロジェクト', [125.0, 175.0, 240.0, 175.0, 240.0, 188.0, 125.0, 188.0]],
 ['₸100-0001', [397.0, 175.0, 459.0, 176.0, 459.0, 188.0, 397.0, 187.0]],
 ['2022/4/30', [123.0, 195.0, 182.0, 195.0, 182.0, 207.0, 123.0, 207.0]],
 ['東京都千代田区千代田1-1-1', [397.0, 194.0, 543.0, 194.0, 543.0, 208.0, 397.0, 208.0]],
 ['支払条件', [56.0, 213.0, 104.0, 214.0, 104.0, 227.0, 56.0, 2

In [35]:
result = result_list[1]

In [36]:
result

['Conference room 154584354',
 [131.0, 153.0, 224.0, 153.0, 224.0, 161.0, 131.0, 160.0]]

In [38]:
result[1][0]

131.0

In [44]:
name = '001_見積書_タテ型_通常版.pdf'

In [47]:
name.split('.')[-1]

'pdf'

In [48]:
name

'001_見積書_タテ型_通常版.pdf'

In [49]:
import pdfplumber

In [50]:
import pdfplumber
from PIL import Image

pdf_path = "./001_見積書_タテ型_通常版.pdf"
output_path = "./image.png"  # 出力先のパス

with pdfplumber.open(pdf_path) as pdf:
    first_page = pdf.pages[0]  # 最初のページを取得
    im = first_page.to_image()  # 画像に変換
    im.save(output_path, format="PNG")  # 画像として保存